In [ ]:
from joblib import load
pca = load('pca_model_8_0206.joblib')

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
import torch.nn.functional as F
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import numpy as np
# load monomers_info
monomers_info_df = pd.read_csv('monomers_info.csv')
monomers_info_df.set_index('Unnamed: 0', inplace=True)
#columns_to_keep = ['MW', 'complexity', 'dft_sp_E_RB3LYP', 'solubility_sqrt_MJperm3']
#monomers_info_df = monomers_info_df[columns_to_keep]

scaler = MinMaxScaler()
monomers_info_df_normalized = pd.DataFrame(scaler.fit_transform(monomers_info_df), columns=monomers_info_df.columns, index=monomers_info_df.index)

merged_df = pd.read_csv('merged_df_PCA8_0206_e_value3.csv')

for monomer in ['R1(HA)', 'R2(IA)', 'R3(NVP)', 'R4(AA)', 'R5(HEAA)', 'R6(IBOA)']:
    for feature in monomers_info_df_normalized.columns:
        merged_df[f'{monomer}_{feature}'] = merged_df[monomer] * monomers_info_df.loc[monomer, feature]

In [ ]:


filenames = merged_df['filename']
merged_df.drop(columns=['filename'], inplace=True)

X_columns = [col for col in merged_df.columns if col not in ['PC1', 'PC2', 'PC3', 'PC4', 'PC5', 'PC6', 'PC7', 'PC8',"sample"]]
y_columns = ['PC1', 'PC2', 'PC3', 'PC4', 'PC5', 'PC6', 'PC7', 'PC8']

In [ ]:
from sklearn.model_selection import train_test_split


'''
filename_list = [
    "58_3", "73_2", "79_1", "80_2", "95_1", "109_2",
    "66_2", "72_3", "92_2", "99_2", "105_3", "67_2", "74_3"
]
'''

pattern = '|'.join(filename_list)
test_mask = filenames.str.contains(pattern)
train_mask = ~test_mask
X_train = merged_df.loc[train_mask, X_columns]
y_train = merged_df.loc[train_mask, y_columns]
X_test = merged_df.loc[test_mask, X_columns]
y_test = merged_df.loc[test_mask, y_columns]
filenames_test = filenames[test_mask]

X_train = torch.FloatTensor(X_train.values)

X_test = torch.FloatTensor(X_test.values)
y_test = torch.FloatTensor(y_test.values)


In [ ]:
from torch.utils.data import TensorDataset, DataLoader
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(85, 200),
            nn.LeakyReLU(),
            nn.Dropout(), #dropout rate 
            nn.Linear(200, 100),
            nn.LeakyReLU(),
            nn.Dropout(0.05),
            nn.Linear(100, 50),
            nn.LeakyReLU(),
            nn.Linear(50, 25),
            nn.LeakyReLU(),
            #nn.Dropout(),
            nn.Linear(25, 8)
        )
    
    def forward(self, x):
        return self.layers(x)






model = MLP().to(device) 

optimizer = optim.Adam(model.parameters(), lr=0.001)

batch_size = 32
train_dataset = TensorDataset(X_train, y_train)

train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
# WMSE #bad
#weights = torch.FloatTensor([0.9, 0.1, 0.05, 0.05,0.05,0.01,0.01,0.01])
#weights = torch.FloatTensor([0.9,0.1, 3.43091913e-03*3, 4.89525388e-04,
# 9.90290160e-05, 2.34845799e-05, 1.08903900e-05, 5.65203313e-06])

mse_loss = nn.MSELoss()
def weighted_mse_loss(input, target):
    return torch.sum(weights * (input - target) ** 2)

loss_values = []

epochs = 1001
for epoch in range(epochs):
    model.train()
    for X_batch, y_batch in train_loader:  
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()
    
        output = model(X_batch)  
    
        loss = weighted_mse_loss(output, y_batch)  
        l1_lambda = 0.1

        
        l1_loss = sum(p.abs().sum() for p in model.parameters())
        
        total_loss = loss + l1_lambda * l1_loss
        total_loss.backward()
    
        optimizer.step()
    
        loss_values.append(loss.item())
    
    if epoch % 100 == 0:
        print(f'Epoch {epoch}, Loss: {loss.item()}')


In [ ]:
import numpy as np
import scipy.stats as stats
from sklearn.metrics import r2_score

def calculate_statistics(y_true, y_pred):
    errors = np.array(y_true) - np.array(y_pred)
    mse = np.mean(errors ** 2)
    mae = np.mean(np.abs(errors))
    max_value = np.max(y_true)
    min_value = np.min(y_true)
    range_value = max_value - min_value
    average = np.mean(y_true)

    # STD
    mse_se = stats.sem(errors ** 2)
    mae_se = stats.sem(np.abs(errors))

    # 95% CI
    z_score = 1.96
    mse_ci = z_score * mse_se
    mae_ci = z_score * mae_se
    #  R^2
    r2 = r2_score(y_true, y_pred)
    return mae, mae_ci, mse, mse_ci, max_value, min_value, range_value, average,r2

In [ ]:
# build empty model
model = MLP(drop1=0.1, cellsize=200)  

# load back saved model
model_path = 'saved_models_MSEloss/....pth'

model.load_state_dict(torch.load(model_path))

model.eval()


In [ ]:
model.eval()
with torch.no_grad():
    y_pred = model(X_test)


y_test_np = y_test.numpy()
y_pred_np = y_pred.numpy()

y_test_reconstructed = pca.inverse_transform(y_test_np)
y_pred_reconstructed = pca.inverse_transform(y_pred_np)

y_test_reconstructed = y_test_reconstructed.reshape(-1, 50, 2)
y_pred_reconstructed = y_pred_reconstructed.reshape(-1, 50, 2)

In [ ]:
import torch
from sklearn.metrics import r2_score


# MSE
mse = torch.mean((torch.tensor(y_test_np) - torch.tensor(y_pred_np)) ** 2).item()

# R2
r2 = r2_score(y_test_np, y_pred_np)

print("Mean Squared Error (MSE):", mse)
print("R-squared (R2):", r2)

In [ ]:
from sklearn.metrics import r2_score

r2_scores = []
for i in range(y_test_np.shape[1]): 
    r2 = r2_score(y_test_np[:, i], y_pred_np[:, i])
    r2_scores.append(r2)

# R2 for every PC
for i, score in enumerate(r2_scores):
    print(f"R2 for feature {i}: {score:.2f}")


In [ ]:
def calculate_statistics(y_true, y_pred):
    errors = np.array(y_true) - np.array(y_pred)
    rmse = np.sqrt(np.mean(errors ** 2))
    
    log_errors = np.log(errors**2 + 1e-8) 
    log_rmse_se = stats.sem(log_errors)
    z_score = 1.96
    log_rmse_ci = z_score * log_rmse_se
    rmse_ci_lower, rmse_ci_upper = np.exp(np.log(rmse) + np.array([-1, 1]) * log_rmse_ci)

    mae = np.mean(np.abs(errors))
    mae_se = stats.sem(np.abs(errors))
    mae_ci = z_score * mae_se

 
    max_value = np.max(y_true)
    min_value = np.min(y_true)
    range_value = max_value - min_value
    average = np.mean(y_true)
    r2 = r2_score(y_true, y_pred)

  
    return mae, mae_ci, rmse, (rmse_ci_lower, rmse_ci_upper), max_value, min_value, range_value, average, r2


def print_statistics(name, stats):
    mae, mae_ci, rmse, rmse_ci, max_val, min_val, range_val, avg_val, r2 = stats
    print(f"{name} - MAE: {mae} ± {mae_ci}, RMSE: {rmse} ± {rmse_ci}, R2: {r2}, Max: {max_val}, Min: {min_val}, Range: {range_val}, Average: {avg_val}")

# Example usage
# Note: Replace 'y_true_last_but_one_stats' and 'y_pred_last_but_one_stats' with your actual data
print_statistics("Y Value at Second Last Point", calculate_statistics(y_true_last_but_one_stats, y_pred_last_but_one_stats))
print_statistics("Area Under Curve", calculate_statistics(true_area_stats, pred_area_stats))
# Uncomment and replace the following lines with actual data as needed
# print_statistics("Soft Material Max Stress", calculate_statistics(soft_material_stress_true, soft_material_stress_pred))
# print_statistics("End Slope", calculate_statistics(end_slope_stats_true, end_slope_stats_pred))
